In [55]:
import sys
from model.lightgbm_model import LightGbmModel

import os
import pandas as pd
import pickle
from config import Config
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from gensim.models.word2vec import Word2Vec

from keras.preprocessing import sequence
from keras.utils import np_utils


import neologdn
import os
import urllib.request
from matplotlib.font_manager import FontProperties
import MeCab
from __future__ import unicode_literals
import re
import unicodedata
import sagemaker
from sagemaker import get_execution_role
import boto3
import json
import pandas as pd
import warnings
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import codecs
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#chaneg dummies to matrix
from scipy.sparse import csr_matrix,coo_matrix

import multiprocessing

from multiprocessing import Pool

import pickle

import math
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import  roc_auc_score,f1_score

from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate,Conv1D, MaxPool1D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau,Callback
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
import os
import tarfile
import string
import numpy as np

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas(desc="hoge progress: ")

import time
import gensim
tqdm.pandas()

import numpy as np
import math

from sudachipy import tokenizer
from sudachipy import dictionary


tokenizer_obj = dictionary.Dictionary().create()

import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import KFold
import time
import os
import pickle
import sys
sys.path.append('..')
from config import Config

from sklearn.calibration import CalibratedClassifierCV
from sklearn import svm

In [56]:
!sudachipy link -t full
mode = tokenizer.Tokenizer.SplitMode.A
def wakati_by_sudachi(txt):
    processed=[m.normalized_form() for m in tokenizer_obj.tokenize(txt, mode)]
    return " ".join(processed)
    

unlinking sudachidict
sudachidict unlinked
default dict package = sudachidict_full


In [57]:
def remove_chars(row):
    cust_list=['\d+','■','\n','#','◇','①','②','③','④','【】','】','【','☆','_','%','「','」','★','/']
    del_list = string.ascii_letters+'"#$%&\'()*+,-./:;<=>@[\\]^_`{|}~'
    for i in del_list:
        row = row.str.replace(i,'')
    for i in cust_list :
        row = row.str.replace(i,'')
    return row

In [58]:
def clean_contractions(text):
    mapping ={"矢張り":"やはり","迚も":"とても","迚も":"とても","此れ":"これ","其れ":"それ","此の":"この","此の":"この","可成":"かなり","兎に角":"とにかく","態々":"わざわざ"
             ,"です":"","だ":"","ます":"","て":"","た":""}
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [59]:
def load_comment_data():
    return pd.read_csv('/home/ec2-user/SageMaker/Notebooks_For_CX_Usecases/Sentiment_Analyzer/raw_data/sum_data_for_sentiment.csv')

In [60]:
def cal_acc(pred, label):
    right = 0
    total = 0
    for idx, p in enumerate(pred):
        total += 1
        flag = np.argmax(p)
        if int(flag) == int(label[idx]):
            right += 1
    return right / total

def cal_f_alpha(pred, label, alpha=1.0, n_out=45, verbose=False):
    # pred:  (x, 45)
    # label: (x, 1)
    # matrix = np.zeros((n_out, n_out))
    matrix = np.diag(np.array([1e-7] * n_out))
    for idx, p in enumerate(pred):
        true_label = int(label[idx])
        p = int(np.argmax(p))
        if p == true_label:
            matrix[p][p] += 1
        else:
            matrix[true_label][p] += 1

    pi = []
    ri = []
    for i in range(n_out):
        pi.append(matrix[i][i] / sum(matrix[:, i]) / n_out)
        ri.append(matrix[i][i] / sum(matrix[i, :]) / n_out)

    p = sum(pi)
    r = sum(ri)
    f = (alpha**2 + 1) * p * r / (alpha ** 2 * p + r)
    if verbose:
        # check every categories' prediction and recall
        pass
    return f

In [61]:
df_mutilabel=load_comment_data()

In [62]:
df_mutilabel['comment'] = remove_chars(df_mutilabel['comment'])
df_mutilabel['comment'].replace('', np.nan, inplace=True)
df_mutilabel.dropna(inplace=True)
df_mutilabel['comment']=df_mutilabel['comment'].apply(neologdn.normalize)
pool = Pool(processes=multiprocessing.cpu_count())
df_mutilabel['comment'] = pool.map(wakati_by_sudachi,df_mutilabel['comment'])
pool.close() 
pool.join()


pool = Pool(processes=multiprocessing.cpu_count())
df_mutilabel['comment'] = pool.map(clean_contractions,df_mutilabel['comment'])
pool.close() 
pool.join()
df_mutilabel['comment'].replace('', np.nan, inplace=True)
df_mutilabel.dropna(inplace=True)
df_mutilabel.fillna(' ',inplace=True)

In [63]:
msk = np.random.rand(len(df_mutilabel)) < 0.8
train=df_mutilabel[msk]
test=df_mutilabel[~msk]

In [64]:
train.reset_index(inplace=True,drop=True)
test.reset_index(inplace=True,drop=True)

In [65]:
vec = TfidfVectorizer(ngram_range=(1,wins), min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
kfold_x_train = train[column]
k_trn_term_doc = vec.fit_transform(kfold_x_train)

In [66]:
config = Config()
n_folds = 5
n_class = 2
kf = KFold(n_splits=n_folds, shuffle=True, random_state=10)
column = "comment"
# column = "article"
oof_predict = np.zeros((len(train[column]), n_class))
predict = np.zeros((len(test[column]), n_class))
cur_kfold = 0
accs = []
f1s = []
wins = 3

In [67]:
#with LogisticRegression
for train_index, test_index in kf.split(train[column]):
    vec = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
    kfold_x_train = train[column][train_index]
    kfold_x_valid = train[column][test_index]
    k_y_train = (train['label']).astype(int)[train_index]
    k_y_valid = (train['label']).astype(int)[test_index]

    #print('获得tfidf特征')
    k_trn_term_doc = vec.fit_transform(kfold_x_train)
    k_test_term_doc = vec.transform(kfold_x_valid)
    test_term_doc = vec.transform(test[column])

    # 拟合数据
    #print('拟合数据')
    lin_clf = LogisticRegression(C=4, dual=True)
    lin_clf.fit(k_trn_term_doc, k_y_train)

    # 预测结果
    #print('预测结果')
    oof_predict[test_index] = lin_clf.predict_proba(k_test_term_doc)
    predict += lin_clf.predict_proba(test_term_doc) / n_folds

    # 计算准确度
    accuracy = cal_acc(oof_predict[test_index], k_y_valid.values)
    f1 = cal_f_alpha(oof_predict[test_index], k_y_valid.values, n_out=n_class)
    #print('Test acc = %f\n' % accuracy)
    #print('Test f1 = %f\n' % f1)
    accs.append(accuracy)
    f1s.append(f1)
    cur_kfold += 1

print('total acc scores is ', np.mean(accs))
print('total f1 scores is ', np.mean(f1s))

total acc scores is  0.9443220240413062
total f1 scores is  0.8680432984067095


In [68]:
#with CalibratedClassifierCV
for train_index, test_index in kf.split(train[column]):
    vec = TfidfVectorizer(ngram_range=(1,wins), min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
    kfold_x_train = train[column][train_index]
    kfold_x_valid = train[column][test_index]
    k_y_train = (train['label']).astype(int)[train_index]
    k_y_valid = (train['label']).astype(int)[test_index]

    #print('获得tfidf特征')
    k_trn_term_doc = vec.fit_transform(kfold_x_train)
    k_test_term_doc = vec.transform(kfold_x_valid)
    test_term_doc = vec.transform(test[column])

    # 拟合数据
    #print('拟合数据')
    lin_clf = svm.LinearSVC()
    lin_clf = CalibratedClassifierCV(lin_clf)
    lin_clf.fit(k_trn_term_doc, k_y_train)

    # 预测结果
    #print('预测结果')
    oof_predict[test_index] = lin_clf.predict_proba(k_test_term_doc)
    predict += lin_clf.predict_proba(test_term_doc) / n_folds

    # 计算准确度
    accuracy = cal_acc(oof_predict[test_index], k_y_valid.values)
    f1 = cal_f_alpha(oof_predict[test_index], k_y_valid.values, n_out=n_class)
    #print('Test acc = %f\n' % accuracy)
    #print('Test f1 = %f\n' % f1)
    accs.append(accuracy)
    f1s.append(f1)
    cur_kfold += 1

print('total acc scores is ', np.mean(accs))
print('total f1 scores is ', np.mean(f1s))

total acc scores is  0.9460637713260814
total f1 scores is  0.8732405856512834
